<h1>Demonstration Notebook: Simulation of reflected and stopped Brownian motion in a wedge</h1>

In [1]:
import numpy as np
from simulationV3 import *

We first simulate a Brownian motion which is stopped or reflected in a wedge.

The object <tt>BrownianMotion</tt> has several methods; to apply a Monte Carlo simulation on one of these methods:

<tt>BrownianMotion.Monte_Carlo(F_test, method, N, *args)</tt>

where <tt>N</tt> is the number of Monte Carlo simulations and where <tt>*args</tt> are the arguments of the <tt>method</tt>. The different methods are:
<ul>
    <li><tt>stopped_BM(self, stopAtT = True, computeTau = True)</tt>: simulate the stopped Brownian motion; if <tt>stopAtT=False</tt>, then we take $T=\infty$ and stop at $\tau$ instead of $\min(\tau, T)$.</li>
    <li><tt>metzler_stopped(self, computeTau = False, N_approx=25)</tt>: simulate the stopped Brownian motion according to Metzler's method.</li>
    <li><tt>reflected_BM(self, approx = False, epsilon = 0., stopNbIter = False, nb_iter_MAX = 200)</tt>: simulate the reflected Brownian motion; <tt>approx</tt> is whether to use the approximation algorithm with parameter <tt>epsilon</tt>. If <tt>stopNbIter=True</tt>, then the algorithm is stopped after <tt>nb_iter_Max</tt> iterations, thus preventing infinite running time.</li>
</ul>

In [2]:
# parameters
alpha = 0.9
r0 = 1.5
theta0 = alpha/3.
T = 1.

def F_test(x):
    return np.sum(x**2)

BM = BrownianMotion(r0, theta0, T, alpha)

# Simulate f(W_tau); both should give the same result
# the last arguments are the arguments of the method called
result = BM.Monte_Carlo(F_test, "metzler_stopped", 50000, False)
result = BM.Monte_Carlo(F_test, "stopped_BM", 50000, False, False)

# Simulate tau, use computeTau=True
result = BM.Monte_Carlo(F_test, "metzler_stopped", 10000, True, 15)
result = BM.Monte_Carlo(F_test, "stopped_BM", 10000, False)

# Simulate f(W_(min(tau,T)))
result = BM.Monte_Carlo(F_test, "stopped_BM", 10000)

# Simulate the reflected Brownian motion
# The first case is the exact algorithm, which has infinite mean running time
# The second one is the approximation algorithm with epsilon=1e-2
result = BM.Monte_Carlo(F_test, "reflected_BM", 100)
result = BM.Monte_Carlo(F_test, "reflected_BM", 10000, True, 1e-2)

Expectation is   3.4476 +-   0.0641 with simulation time   1.0971 and average number of iterations   0.0000
Simulated variance is  53.4881
Simulated average min(tau,T) is   0.0000 +-   0.0000

Expectation is   3.4657 +-   0.0777 with simulation time   1.5580 and average number of iterations   1.4529
Simulated variance is  78.5044
Simulated average min(tau,T) is   0.0000 +-   0.0000

Expectation is   3.3383 +-   0.1183 with simulation time   3.8167 and average number of iterations   0.0000
Simulated variance is  36.4348
Simulated average min(tau,T) is   0.7343 +-   0.0361

Expectation is   3.3991 +-   0.1440 with simulation time   2.2683 and average number of iterations   1.4528
Simulated variance is  53.9892
Simulated average min(tau,T) is   0.6154 +-   0.0430

Expectation is   3.0276 +-   0.0517 with simulation time   6.5220 and average number of iterations   1.3832
Simulated variance is   6.9599
Simulated average min(tau,T) is   0.3901 +-   0.0062

Expectation is   4.2034 +-   0.6434

The results of the Monte Carlo simulation are given in a dictionary where
<ul>
    <li><tt>E1</tt> is the simulated expectation.</li>
    <li><tt>interval</tt> is the 95% confidence interval.</li>
    <li><tt>E_nb_iter</tt> is the average number of iterations.</li>
    <li><tt>nb_iter_list</tt> is the list of the number of iterations for each Monte Carlo simulation.</li>
    <li><tt>E_T_n</tt> is the average value of $\min(\tau, T)$ if $\tau$ is computed; otherwise returns 0.</li>
    <li><tt>interval_T_n</tt> is the 95% confidence interval for $\min(\tau, T)$.</li>
</ul>

We now simulate a process which is stopped or reflected in a wedge. For now, only the case $\sigma = I_2$ is available.

In [3]:
# parameters
alpha = 0.9
r0 = 1.5
theta0 = alpha/3.
T = 1.

x0 = np.array([r0*np.cos(theta0), r0*np.sin(theta0)])
mu = np.array([0.1, 0.2])
a = np.array([0.7, 0.5])

# drift
def b(x):
    return -mu*(x-a)

def F_test(x):
    return np.sum(x**2)

# stopped process
# the last argument is the number of steps in the Euler scheme
process = StoppedProcess(x0, T, alpha, b, 100)
result = process.Monte_Carlo(F_test, 1000)

# reflected process
process = ReflectedProcess(x0, T, alpha, b, 100, epsilon=1e-2)
result = process.Monte_Carlo(F_test, 1000)

Expectation is   2.8326 +-   0.1469 with simulation time  20.0309
Simulated variance is   5.6179

Expectation is   3.8841 +-   0.2016 with simulation time  52.1819
Simulated variance is  10.5843



The results of <tt>Monte_Carlo</tt> are given in a dictionnary as before, however only <tt>E1</tt> and <tt>interval</tt> are given.

We now check that if the drift $b \equiv 0$ then both methods return the same results.

In [4]:
def b(x):
    return np.zeros(2)

result = BM.Monte_Carlo(F_test, "stopped_BM", 10000)
result = StoppedProcess(x0, T, alpha, b, 100).Monte_Carlo(F_test, 1000)

result = BM.Monte_Carlo(F_test, "reflected_BM", 5000, True, 1e-2)
result = ReflectedProcess(x0, T, alpha, b, 100, epsilon=1e-2).Monte_Carlo(F_test, 1000)

Expectation is   2.9546 +-   0.0491 with simulation time   6.7682 and average number of iterations   1.3845
Simulated variance is   6.2712
Simulated average min(tau,T) is   0.3874 +-   0.0062

Expectation is   3.0666 +-   0.1623 with simulation time  20.2761
Simulated variance is   6.8610

Expectation is   4.2382 +-   0.1014 with simulation time  14.6793 and average number of iterations   5.6866
Simulated variance is  13.3861
Simulated average min(tau,T) is   0.9865 +-   0.0022

Expectation is   4.2395 +-   0.2196 with simulation time  51.9240
Simulated variance is  12.5585

